In [ ]:
from fastai.vision.all import *
import pandas as pd

In [ ]:
!mkdir -p  /tmp/.torch/models/
!cp ../input/resnet50/resnet50.pth  /tmp/.torch/models/resnet50-19c8e357.pth

In [ ]:
import torchvision.models
from torchvision.models.resnet import _resnet, resnet50, Bottleneck

def _resnet(arch, block, layers, pretrained, progress, **kwargs):
    model = ResNet(block, layers, **kwargs)
    if pretrained:
        state_dict = torch.load('/tmp/.torch/models/resnet50-19c8e357.pth')
        model.load_state_dict(state_dict)
    return model

def resnet50(pretrained=False, progress=True, **kwargs):
    return _resnet('resnet50', Bottleneck, [3, 4, 6, 3], pretrained, progress, **kwargs)

In [ ]:
path = Path('../input/cassava-leaf-disease-classification/')
train = pd.read_csv(path/'train.csv')

In [ ]:
dls = ImageDataLoaders.from_df(train, path=path, valid_pct=0.1, seed=42, folder='train_images', bs=64,
                              item_tfms=Resize(224), batch_tfms=aug_transforms())

In [ ]:
dls.show_batch()

In [ ]:
learn = cnn_learner(dls, resnet50, metrics=accuracy)

In [ ]:
learn.fine_tune(5)

In [ ]:
test_images = get_image_files(path/'test_images')
fnames = [f.name for f in test_images]

In [ ]:
tst_dl = dls.test_dl(test_images)
_,_,preds = learn.get_preds(dl=tst_dl, with_decoded=True)

In [ ]:
sub = pd.DataFrame(fnames, columns=['image_id'])
sub['label'] = preds

In [ ]:
sub.to_csv('submission.csv', index=False)

In [ ]:
sub.head()